In [1]:
from src.argilla.server.daos.backend.generic_elastic import GenericElasticEngineBackend

In [2]:
elastic_backend = GenericElasticEngineBackend.get_instance()

In [3]:
elastic_backend.client

ElasticsearchClient(vector_search_supported=True, index_shards=1, config_backend={'hosts': 'http://localhost:9200', 'verify_certs': True, 'ca_certs': None, 'retry_on_timeout': True, 'max_retries': 5})

In [4]:
%pip install faiss-cpu sentence_transformers transformers datasets

Note: you may need to restart the kernel to use updated packages.


In [7]:
from datasets import load_dataset

agnews = load_dataset("ag_news")

agnews_train, agnews_valid = (
    agnews["train"].train_test_split(test_size=7600, seed=43).values()
)


2022-11-24 18:46:20.136 | WARNING  | datasets.builder:_create_builder_config:412 - Using custom data configuration default
2022-11-24 18:46:20.141 | WARNING  | datasets.builder:download_and_prepare:577 - Reusing dataset ag_news (/home/ufuk/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


  0%|          | 0/2 [00:00<?, ?it/s]

2022-11-24 18:46:20.216 | WARNING  | datasets.arrow_dataset:train_test_split:3426 - Loading cached split indices for dataset at /home/ufuk/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548/cache-0fb8fdf64cc3fcc4.arrow and /home/ufuk/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548/cache-80423eae5320d3cb.arrow


In [ ]:
import argilla as rg

# build our labelled records to evaluate our heuristic rules and optimize the thresholds
records = [
    rg.TextClassificationRecord(
        text=record["text"],
        metadata={"split": "labelled"},
        annotation=agnews_valid.features["label"].int2str(record["label"]),
        id=f"valid_{idx}",
    )
    for idx, record in enumerate(agnews_valid)
]

# build our unlabelled records
records += [
    rg.TextClassificationRecord(
        text=record["text"],
        metadata={"split": "unlabelled"},
        id=f"train_{idx}",
    )
    for idx, record in enumerate(agnews_train)
]

# log the records to Argilla
rg.log(records, name="news2")

BaseClientError: Your Api endpoint at http://localhost:6900 is not available or not responding.